In [1]:
#Doing the common things in bow and this model first
import nltk
import re
import heapq
import numpy as np

paragraph = """Thank you all so very much. Thank you to the Academy. 
               Thank you to all of you in this room. I have to congratulate 
               the other incredible nominees this year. The Revenant was 
               the product of the tireless efforts of an unbelievable cast
               and crew. First off, to my brother in this endeavor, Mr. Tom 
               Hardy. Tom, your talent on screen can only be surpassed by 
               your friendship off screen … thank you for creating a t
               ranscendent cinematic experience. Thank you to everybody at 
               Fox and New Regency … my entire team. I have to thank 
               everyone from the very onset of my career … To my parents; 
               none of this would be possible without you. And to my 
               friends, I love you dearly; you know who you are. And lastly,
               I just want to say this: Making The Revenant was about
               man's relationship to the natural world. A world that we
               collectively felt in 2015 as the hottest year in recorded
               history. Our production needed to move to the southern
               tip of this planet just to be able to find snow. Climate
               change is real, it is happening right now. It is the most
               urgent threat facing our entire species, and we need to work
               collectively together and stop procrastinating. We need to
               support leaders around the world who do not speak for the 
               big polluters, but who speak for all of humanity, for the
               indigenous people of the world, for the billions and 
               billions of underprivileged people out there who would be
               most affected by this. For our children’s children, and 
               for those people out there whose voices have been drowned
               out by the politics of greed. I thank you all for this 
               amazing award tonight. Let us not take this planet for 
               granted. I do not take tonight for granted. Thank you so very much."""
               
#preprocessing dataset
# Tokenize sentences
dataset = nltk.sent_tokenize(paragraph)
for i in range(len(dataset)):
    dataset[i] = dataset[i].lower()
    dataset[i] = re.sub(r'\W',' ',dataset[i])
    dataset[i] = re.sub(r'\s+',' ',dataset[i])


# Creating word histogram
word2count = {}
for data in dataset:
    words = nltk.word_tokenize(data)
    for word in words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1
            
# Selecting best 100 features
freq_words = heapq.nlargest(100,word2count,key=word2count.get)

In [2]:
#creating IDF matrix first

word_idfs = {} #creating a dictionary in which each of the different word will me mapped with their idf values

for word in freq_words:
    doc_count = 0 #it contains the number of document containing that specific word
    for data in dataset:
        if word in nltk.word_tokenize(data):
            doc_count += 1; #so if this word appears in any of the document then we are simply incrementing doc_count
    word_idfs[word] = np.log((len(dataset)/doc_count)+1) #here +1 is called a bias
    #note len(dataset) is number of documents in whole corpus
#note in the real word scenerio we don't have the no. of documents  we need to do +1 (will dicuss later in the course)
#note you don't necesserily need to do +1 this is just followed in many of the idf libraries its kind of a standard
print(word_idfs)

{'the': 1.1314021114911006, 'to': 1.067840630001356, 'you': 1.2039728043259361, 'of': 1.5040773967762742, 'for': 1.5040773967762742, 'this': 1.2039728043259361, 'thank': 1.2878542883066382, 'and': 1.3862943611198906, 'i': 1.5040773967762742, 'my': 1.8325814637483102, 'all': 1.8325814637483102, 'in': 2.0794415416798357, 'be': 1.8325814637483102, 'who': 2.4423470353692043, 'world': 2.0794415416798357, 'very': 2.0794415416798357, 'have': 2.0794415416798357, 'by': 2.0794415416798357, 'we': 2.0794415416798357, 'our': 2.0794415416798357, 'is': 2.4423470353692043, 'not': 2.0794415416798357, 'people': 2.4423470353692043, 'out': 2.4423470353692043, 'so': 2.4423470353692043, 'much': 2.4423470353692043, 'year': 2.4423470353692043, 'revenant': 2.4423470353692043, 'was': 2.4423470353692043, 'off': 2.4423470353692043, 'tom': 2.4423470353692043, 'your': 3.091042453358316, 'screen': 3.091042453358316, 'a': 2.4423470353692043, 'entire': 2.4423470353692043, 'would': 2.4423470353692043, 'just': 2.4423470

In [3]:
#making the TF matrix
tf_matrix = {}  #it is contain all the frequent words and corresponding to those words we will store a vector of
#length 21 (because there are 21 sentences in the whole dataset). So we are going to calculate the TF of each of the
#word in all of these 21 sentences. so we are going to gave a vector of different numbers
#note A vector in a simple term can be considered as a single-dimensional array.
#i.e., a vector is a one-dimensional array of lists.
for word in freq_words: #looping through frequent words
    doc_tf = [] #vector for each of the word initializing vector
    for data in dataset: #looping through each sentence/document
        frequency = 0 #initializing frequency to 0 for each of the document
        for w in nltk.word_tokenize(data): #tokenizing each of the document into different words
            if w == word: #checking if this word is equal to our frequent word
                frequency += 1 #incrementing frequency by 1
        #now we will calculate tf of that word in this specific document
        tf_word = frequency/len(nltk.word_tokenize(data)) #note here we have to tokenize again because not in loop
        doc_tf.append(tf_word)
    tf_matrix[word] = doc_tf

print(tf_matrix)
#here we have a list or a vector of size 21 So it has 21 size list for each of the corresponding keys(frequent words)

{'the': [0.0, 0.2, 0.0, 0.1, 0.2, 0.0, 0.0, 0.0, 0.043478260869565216, 0.0, 0.1, 0.06666666666666667, 0.05263157894736842, 0.0, 0.05, 0.10638297872340426, 0.045454545454545456, 0.0, 0.0, 0.0, 0.0], 'to': [0.0, 0.2, 0.1111111111111111, 0.1, 0.0, 0.09090909090909091, 0.0, 0.08333333333333333, 0.08695652173913043, 0.07692307692307693, 0.1, 0.0, 0.21052631578947367, 0.0, 0.05, 0.02127659574468085, 0.0, 0.0, 0.0, 0.0, 0.0], 'you': [0.16666666666666666, 0.2, 0.2222222222222222, 0.0, 0.0, 0.0, 0.043478260869565216, 0.08333333333333333, 0.043478260869565216, 0.23076923076923078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1111111111111111, 0.0, 0.0, 0.2], 'of': [0.0, 0.0, 0.1111111111111111, 0.0, 0.13333333333333333, 0.0, 0.0, 0.0, 0.08695652173913043, 0.0, 0.0, 0.0, 0.05263157894736842, 0.0, 0.0, 0.06382978723404255, 0.045454545454545456, 0.0, 0.0, 0.0, 0.0], 'for': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.043478260869565216, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0851063829787234, 0.09090909090909091, 0

In [4]:
#now we will calculate the TF_IDF scores for each word in each document
tfidf_matrix = [] #using a list instead of dictionary we can use dictionary also
for word in tf_matrix.keys(): 
    tfidf = [] #its going to contain all the TF-IDF scores for specific words
    #note the value stored corresponding to word in tf_matrix dictionary is a list so we are looping through it
    for value in tf_matrix[word]:
        score = value*word_idfs[word] #applying formulla
        tfidf.append(score)
    tfidf_matrix.append(tfidf)
    
print(tfidf_matrix)
#note it is containg 100 list and each list is containg 21 tfidf values of 21 corresponding documents

[[0.0, 0.22628042229822012, 0.0, 0.11314021114911006, 0.22628042229822012, 0.0, 0.0, 0.0, 0.049191396151786984, 0.0, 0.11314021114911006, 0.07542680743274004, 0.059547479552163184, 0.0, 0.05657010557455503, 0.1203619267543724, 0.051427368704140934, 0.0, 0.0, 0.0, 0.0], [0.0, 0.21356812600027122, 0.11864895888903956, 0.10678406300013561, 0.0, 0.09707642090921419, 0.0, 0.08898671916677967, 0.09285570695663965, 0.08214158692318124, 0.10678406300013561, 0.0, 0.224808553684496, 0.0, 0.053392031500067806, 0.022720013404284173, 0.0, 0.0, 0.0, 0.0, 0.0], [0.20066213405432268, 0.24079456086518725, 0.2675495120724302, 0.0, 0.0, 0.0, 0.05234664366634505, 0.10033106702716134, 0.05234664366634505, 0.2778398779213699, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1337747560362151, 0.0, 0.0, 0.24079456086518725], [0.0, 0.0, 0.16711971075291934, 0.0, 0.2005436529035032, 0.0, 0.0, 0.0, 0.1307893388501108, 0.0, 0.0, 0.0, 0.07916196825138284, 0.0, 0.0, 0.09600494021976218, 0.06836715439892156, 0.0, 0.0, 0.0, 0.0]

In [5]:
#note it is still not done we have to transpose it and convert it into a 2d array to make it ready to feed to any
#ml algorithm

#convering this to a 2d array
X = np.asarray(tfidf_matrix)
print(X)

[[0.         0.22628042 0.         ... 0.         0.         0.        ]
 [0.         0.21356813 0.11864896 ... 0.         0.         0.        ]
 [0.20066213 0.24079456 0.26754951 ... 0.         0.         0.24079456]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [6]:
#now we have to transpose it to make it ready for any ml algo
#now it is 100 rows and 21 columns
#after transposing we will get it as 21 rows and 100 columns
X = np.transpose(X)
print(X)

[[0.         0.         0.20066213 ... 0.         0.         0.        ]
 [0.22628042 0.21356813 0.24079456 ... 0.         0.         0.        ]
 [0.         0.11864896 0.26754951 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.24079456 ... 0.         0.         0.        ]]
